# Foreign Exchange Return Forecasting of Neighboring Countries based on Powerful Anchor Countries
# *SIADS 696: Milestone II*

### *By Team #2*

## Overview

*Our team studied the impact on foreign exchange returns between neighboring countries and anchoring countries (neighboring countries with a larger GDP). We studied:*

`H_0` (null): Anchoring country foreign exchange rates have no impact on foreign exchange return rates of neighboring countries.

`H_1` (alternative): Anchoring country foreign exchange rates have impact on foreign exchange return rates of neighboring countries.


## Setup

```
.
├── data
│   ├── input
│   └── output
├── docs
├── logs
└── src


## Sections
- Defining Custom Functions
- Importing Data
- Applying Unsupervised Learning
- Applying Supervised Learning
    - Grangier Causality
    - Linear Regression
    - XGBoost
    - KNN
- Testing Model Performance
    - Performing Ablation
    - Performing Feature Importance
- Visualization
- Conclusion


## Setup

In [1]:
# Importing Packages
import country_converter as coco
import matplotlib.pyplot as plt
import xgboost as xgb
import yfinance as yf
import pandas as pd
import numpy as np
import warnings
import csv
import re

# Importing partial packages
from sklearn.metrics import mean_squared_error, silhouette_score, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from utility import create_lag, load_data
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from xgboost import XGBClassifier

In [2]:
# Ignoring unnecessary warnings
warnings.filterwarnings("ignore")

# Showing all rows
# pd.set_option('display.max_rows', 100)

## Defining Custom Functions

In [ ]:
# Preprocessing column names to standardize
def standardize_column_names(dataframe):
    dataframe.columns = [
        str(column).lower().replace(" ", "_").replace(",", "")
        for column in dataframe.columns
    ]

    return dataframe

In [ ]:
def xxxusd(code):
    x = f"{code}USD=X", f"USD{code}=X"
    return x

In [ ]:
# Preprocessing functions
def clean(country):
    if pd.isna(country):
        return country
    country = str(country).strip()
    country = p.sub("", country)
    return re.sub(r",\s*The$", "", country, flags=re.I)

In [ ]:
# Preprocessing functions
def short(country):
    if pd.isna(country):
        return pd.NA
    res = cc.convert(names=clean(country), to="name_short")
    if isinstance(res, (list, tuple)):
        res = res[0] if res else pd.NA
    if not res or res == "not found" or pd.isna(res):
        warnings.warn(f"Problem: {country}")
        return pd.NA
    return res

In [ ]:
# Defining the Granger causality function
def granger(res, g, ml=5, a=0.05):
    x = []
    for i, j in g.items():
        for k in j:
            if i not in res.columns or k not in res.columns:
                continue
            df = pd.concat([res[k], res[i]], axis=1).dropna()
            df.columns = ["neighbor", "anchor"]
            if len(df) <= ml * 3:
                continue
            t = grangercausalitytests(df, maxlag=ml)
            pval = [t[lag][0]["ssr_ftest"][1] for lag in range(1, ml + 1)]
            min_p = float(np.min(pval))
            best_lag = int(np.argmin(pval) + 1)
            x.append(
                {
                    "anchor": i,
                    "neighbor": k,
                    "min_pval": round(min_p, 4),
                    "best_lag (days)": best_lag,
                    "significant": (min_p < a),
                }
            )
    rdf = pd.DataFrame(x)
    return rdf

In [ ]:
def create_lag(neighbor, dataframe):
    # Create lag features (e.g., previous 5 days' returns)
    for i in range(1, 25):
        dataframe[f"return_lag_{i}"] = dataframe[neighbor].pct_change().shift(i)

    # Create moving average features
    dataframe["MA_30"] = dataframe[neighbor].rolling(window=30).mean().shift(1)
    dataframe["MA_60"] = dataframe[neighbor].rolling(window=60).mean().shift(1)
    dataframe["MA_90"] = dataframe[neighbor].rolling(window=90).mean().shift(1)

    # Drop initial rows with NaNs created by rolling windows
    dataframe.dropna(inplace=True)
    dataframe = dataframe.replace([np.inf, -np.inf], np.nan).fillna(0)

    return dataframe

In [ ]:
# Run this for the neighbor alone: control
# Run this for the anchor + neighbor: treatment
def perform_xgb_for_anchor_neighbor(neighbor, dataframe):
    # Starting with CNY/CDF as an example
    dataframe["target"] = dataframe[neighbor].shift(-1)

    # Define features (X) and target (y)
    features = [
        col
        for col in dataframe.columns
        if col == neighbor or col.startswith("MA_") or col.startswith("return_lag_")
    ]
    X = dataframe[features]
    y = dataframe["target"]

    # Time-based train-test split
    split_index = int(len(X) * 0.8)
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    xgb_0 = xgb.XGBRegressor()
    parameters = {
        "nthread": [4],
        "objective": ["reg:squarederror"],
        "learning_rate": [0.08, 0.15, 0.3],
        "max_depth": [2, 4, 8],
        "min_child_weight": [4],
        "subsample": [0.2, 0.5, 0.8],
        "colsample_bytree": [0.7],
        "n_estimators": [2, 5, 10],
    }

    xgb_grid = GridSearchCV(xgb_0, parameters, cv=2, n_jobs=5, verbose=True)
    xgb_grid.fit(X_train, y_train)

    return xgb_grid.best_score_, xgb_grid.best_params_

In [ ]:
# def visualize_xgb_tree(xgb_grid):

#     # Visualizing the XGBoost model
#     fig, ax = plt.subplots(figsize=(2, 5))
#     xgb.plot_tree(xgb_grid, ax=ax)

#     return plt.show()

## Importing Data

In [ ]:
# Importing Real GDP Purchasing Power Parity
# Note the Ranking; 1 = big country per region
country_gdp = pd.read_csv("../data/input/real_gdp_purchasing_power_parity_0.csv")

# Displaying the first 5 rows of the dataframe
country_gdp.head()

In [ ]:
# Importing Fred Anchors
df_fred_world_gdp = pd.read_csv("../data/input/fred_anchors_0.csv")

# Preprocessing Fred Anchors
df_fred_world_gdp["observation_date"] = pd.to_datetime(
    df_fred_world_gdp["observation_date"]
)
df_fred_world_gdp["time_period"] = df_fred_world_gdp["observation_date"].dt.year
df_fred_world_gdp = df_fred_world_gdp.drop(columns=["observation_date"]).rename(
    columns={"NYGDPMKTPCDWLD": "world_gdp"}
)

# Displaying the first 5 rows of the dataframe
df_fred_world_gdp.head()

In [ ]:
# Importing GDP
df_imf_gdp = pd.read_csv(
    "../data/input/imf_gdp_0.csv",
    on_bad_lines="skip",
    sep=",",
    quoting=csv.QUOTE_ALL,
    skipinitialspace=True,
    usecols=[
        "COUNTRY",
        "TIME_PERIOD",
        "TYPE_OF_TRANSFORMATION",
        "FREQUENCY",
        "OBS_VALUE",
        "INDICATOR",
    ],
    engine="python",
)

# Preprocessing GDP
df_imf_gdp = (
    standardize_column_names(df_imf_gdp)
    .query(
        "indicator == 'US Dollar per domestic currency' "
        "and type_of_transformation == 'End-of-period (EoP)' "
        "and frequency == 'Annual'"
    )
    .dropna(subset=["obs_value"])
    .rename(columns={"time_period": "year"})
    .assign(year=lambda d: d["year"].str[:4])
    .sort_values(["country", "year"])
    .drop(columns=["indicator", "type_of_transformation", "frequency"])
    .reset_index(drop=True)
).assign(year=lambda d: pd.to_datetime(d["year"], errors="coerce").dt.year)

# Displaying the first 5 rows of the dataframe
df_imf_gdp.head()

In [ ]:
# Importing Exchange Rates
df_imf_trade = pd.read_csv(
    "../data/input/imf_trade_0.csv",
    usecols=[
        "COUNTRY",
        "COUNTERPART_COUNTRY",
        "TIME_PERIOD",
        "OBS_VALUE",
        "TRADE_FLOW",
        "SCALE",
        "UNIT",
    ],
)

# Manually excluding countries that are either trade in pegged currencies or don't have a true boundary from another
ls_0 = [
    "World",
    "Advanced Economies",
    "Latin America and the Caribbean (LAC)",
    "Hong Kong Special Administrative Region, People's Republic of China",
    "Emerging and Developing Europe",
    "Middle East and Central Asia",
    "Emerging Market and Developing Economies",
    "Euro Area (EA)",
    "Emerging and Developing Asia",
]

# Excluding the "Exclusion List"
df_imf_trade = df_imf_trade[
    (~df_imf_trade["COUNTRY"].isin(ls_0))
    & (~df_imf_trade["COUNTERPART_COUNTRY"].isin(ls_0))
]

# Viewing first five rows
df_imf_trade = standardize_column_names(df_imf_trade).sort_values(
    by=["obs_value"], ascending=False
)

# Displaying the first 5 rows of the dataframe
df_imf_trade.head()

In [ ]:
# Preprocessing Exchange Rates
cc = coco.CountryConverter()
p = re.compile(
    r",\s*(?:Kingdom of the Netherlands|United Kingdom-British Overseas Territory|Republic of the|Union of the|State of the)$",
    re.I,
)

# Defining countries with pegged currencies
pegged = [
    "Aruba",
    "The Bahamas",
    "Bahrain",
    "Barbados",
    "Belize",
    "Bermuda",
    "Cayman Islands",
    "Cuba",
    "Djibouti",
    "Antigua and Barbuda",
    "Dominica",
    "Grenada",
    "Saint Kitts and Nevis",
    "Saint Lucia",
    "Saint Vincent and the Grenadines",
    "El Salvador",
    "Eritrea",
    "Hong Kong",
    "Jordan",
    "Kuwait",
    "Lebanon",
    "Curaçao",
    "Sint Maarten",
    "Oman",
    "Panama",
    "Qatar",
    "Saudi Arabia",
    "United Arab Emirates",
    "Venezuela",
]

pegged = {short(x) for x in pegged}
pegged.discard(pd.NA)

In [ ]:
# Defining countries in the European Union that use the Euro (EUR) and British Pound (GBP)
EUR = [
    "Austria",
    "Belgium",
    "Cyprus",
    "Estonia",
    "Finland",
    "France",
    "Germany",
    "Greece",
    "Ireland",
    "Italy",
    "Latvia",
    "Lithuania",
    "Luxembourg",
    "Malta",
    "Netherlands",
    "Portugal",
    "Slovakia",
    "Slovenia",
    "Spain",
    "Andorra",
    "Monaco",
    "San Marino",
    "Vatican City",
    "Saint Barthélemy",
    "Saint Pierre and Miquelon",
    "Kosovo",
    "Montenegro",
    "Bosnia and Herzegovina",
    "Bulgaria",
    "Cape Verde",
    "Cameroon",
    "Central African Republic",
    "Chad",
    "Republic of the Congo",
    "Equatorial Guinea",
    "Gabon",
    "Benin",
    "Burkina Faso",
    "Côte d'Ivoire",
    "Guinea-Bissau",
    "Mali",
    "Niger",
    "Senegal",
    "Togo",
    "French Polynesia",
    "New Caledonia",
    "Wallis and Futuna",
    "Comoros",
    "Croatia",
    "Morocco",
    "São Tomé and Príncipe",
    "Denmark",
    "North Macedonia",
]

# Defining countries that use the British Pound (GBP)
GBP = [
    "Guernsey",
    "Jersey",
    "Isle of Man",
    "Gibraltar",
    "Falkland Islands",
    "Saint Helena",
]

# Defining other countries with pegged currencies or no true boundary
other = [
    "Bhutan",
    "Nepal",
    "North Korea",
    "Afghanistan",
    "Turkmenistan",
    "South Sudan",
    "Guam",
    "Macau",
    "Tuvalu",
    "Kiribati",
    "Palau",
    "Greenland",
    "Maldives",
    "Iraq",
    "Solomon Islands",
    "Brunei Darussalam",
    "Bangladesh",
    "Myanmar",
    "Marshall Islands",
    "Iran",
    "Yemen",
    "Libya",
    "Somalia",
    "Liberia",
    "Sudan",
    "Sierra Leone",
    "Mongolia",
    "Angola",
    "Kyrgyz Republic",
    "Tajikistan",
]

# Cleaning country names
EUR = {short(x) for x in EUR if pd.notna(short(x))}
GBP = {short(x) for x in GBP if pd.notna(short(x))}
other = {short(x) for x in other if pd.notna(short(x))}

In [ ]:
# Importing cleaned data
# TODO - where is this coming from?
# I.e. what code created these files?
df_trade = pd.read_parquet("../data/input/trade_0.parquet")
df_gdp = pd.read_parquet("../data/input/gdp_0.parquet")

## Applying Unsupervised Learning

In [ ]:
# Converting country names to short format
cc = coco.CountryConverter()
p = re.compile(
    r",\s*(?:Kingdom of the Netherlands|United Kingdom-British Overseas Territory|Republic of the|Union of the|State of the)$",
    re.I,
)

In [ ]:
# Anchors selection
df_gdp = df_gdp[~(df_gdp["COUNTRY"] == "United States")]
df_gdp.head()
Anchor = {
    "EUR": EUR,
    "CNY": {"China"},
    "JPY": {"Japan"},
}

# No USD due to many relationship ties with multiple currency.
# Time Period
period = 10

# N Neighbor
n = 10

# Minimum Percentage Volume
v = 0.05

# Exports of goods
# Neighbor mapping
neighbor = {x: i for i, y in Anchor.items() for x in y}
ex = EUR | GBP | other | {"China", "Japan", "United States"}
df_trade = df_trade.dropna(
    subset=["COUNTRY", "COUNTERPART_COUNTRY", "TIME_PERIOD", "OBS_VALUE"]
).copy()

# Keep data period
df_trade = df_trade[df_trade["TIME_PERIOD"].astype(str).str.match(r"^\d{4}")]
df_trade["year"] = df_trade["TIME_PERIOD"].astype(str).str[:4].astype(int)
max_year = int(df_trade["year"].max())
df_trade = df_trade[df_trade["year"].between(max_year - period + 1, max_year)]

# Filter and map potential neighbors
df_export = df_trade[~df_trade["COUNTRY"].isin(ex)].copy()
df_export["anchor"] = df_export["COUNTERPART_COUNTRY"].map(neighbor)

# Total exports per country
etot = (
    df_export.groupby("COUNTRY", as_index=False)["OBS_VALUE"]
    .sum()
    .rename(columns={"OBS_VALUE": "export_total"})
)

# Exports to each anchor
eanch = (
    df_export.dropna(subset=["anchor"])
    .groupby(["COUNTRY", "anchor"], as_index=False)["OBS_VALUE"]
    .sum()
    .rename(columns={"OBS_VALUE": "export_to_anchor"})
)

# Export shares
esh = eanch.merge(etot, on="COUNTRY", how="left").assign(
    export_share=lambda d: d["export_to_anchor"] / d["export_total"]
)

# Best export anchor per country
bexp = esh.sort_values(
    ["COUNTRY", "export_share", "export_to_anchor"], ascending=[True, False, False]
).drop_duplicates("COUNTRY")
bexp = bexp[bexp["export_share"] >= v]

# Swap role
df_import = df_trade.rename(
    columns={"COUNTRY": "COUNTERPART_COUNTRY_orig", "COUNTERPART_COUNTRY": "COUNTRY"}
).rename(columns={"COUNTERPART_COUNTRY_orig": "COUNTERPART_COUNTRY"})

# Potential neighbors
df_import = df_import[~df_import["COUNTRY"].isin(ex)].copy()
df_import["anchor"] = df_import["COUNTERPART_COUNTRY"].map(neighbor)

# Total imports per country
import_totals = (
    df_import.groupby("COUNTRY", as_index=False)["OBS_VALUE"]
    .sum()
    .rename(columns={"OBS_VALUE": "import_total"})
)

# Imports from each anchor
ianch = (
    df_import.dropna(subset=["anchor"])
    .groupby(["COUNTRY", "anchor"], as_index=False)["OBS_VALUE"]
    .sum()
    .rename(columns={"OBS_VALUE": "import_from_anchor"})
)

# Import shares
ish = ianch.merge(import_totals, on="COUNTRY", how="left").assign(
    import_share=lambda d: d["import_from_anchor"] / d["import_total"]
)

# Best import anchor per country
bimp = ish.sort_values(
    ["COUNTRY", "import_share", "import_from_anchor"], ascending=[True, False, False]
).drop_duplicates("COUNTRY")
bimp = bimp[bimp["import_share"] >= v]

In [ ]:
# Combine table
comb = esh.merge(ish, on=["COUNTRY", "anchor"], how="outer", suffixes=("_exp", "_imp"))

# Handle missing value
f = [
    "export_to_anchor",
    "export_total",
    "export_share",
    "import_from_anchor",
    "import_total",
    "import_share",
]
for c in f:
    if c not in comb.columns:
        comb[c] = 0.0
comb[f] = comb[f].fillna(0.0)

# Combined metrics
comb = comb.assign(
    total_trade_with_anchor=lambda d: d["export_to_anchor"] + d["import_from_anchor"],
    total_trade_volume=lambda d: d["export_total"] + d["import_total"],
)
comb["combined_exposure"] = 0.0
nz = comb["total_trade_volume"] > 0
comb.loc[nz, "combined_exposure"] = (
    comb.loc[nz, "total_trade_with_anchor"] / comb.loc[nz, "total_trade_volume"]
)
comb_f = comb.loc[comb["combined_exposure"] >= v].copy()

# Rank neighbors per anchor
comb_f = comb_f.sort_values(
    by=[
        "anchor",
        "combined_exposure",
        "total_trade_with_anchor",
        "export_share",
        "import_share",
        "COUNTRY",
    ],
    ascending=[True, False, False, False, False, True],
)

# Rank within each anchor and take top n
comb_f["rank_within_anchor"] = comb_f.groupby("anchor")["combined_exposure"].rank(
    method="first", ascending=False
)
comb_top = comb_f.loc[comb_f["rank_within_anchor"] <= n].copy()

# Final neighbor
neighbors_dict = comb_top.groupby("anchor")["COUNTRY"].apply(list).to_dict()

# Summary results
summary = (
    comb_top[
        [
            "anchor",
            "COUNTRY",
            "combined_exposure",
            "export_share",
            "import_share",
            "export_to_anchor",
            "import_from_anchor",
            "export_total",
            "import_total",
            "total_trade_with_anchor",
            "total_trade_volume",
            "rank_within_anchor",
        ]
    ]
    .sort_values(["anchor", "rank_within_anchor"])
    .reset_index(drop=True)
)

res = summary.copy()
for col in ["combined_exposure", "export_share", "import_share"]:
    res[col] = (res[col] * 100).round(2)

res_sorted = res.sort_values(["anchor", "rank_within_anchor"])
anchor_neighbor = res_sorted.groupby("anchor")["COUNTRY"].apply(list).to_dict()

for a, c in [("CNY", "Australia"), ("JPY", "Philippines"), ("JPY", "Vietnam")]:
    if a in anchor_neighbor and c in anchor_neighbor[a]:
        anchor_neighbor[a].remove(c)

anchor_neighbor

*Performing Principle Component Analyis (PCA) to reduce the dimensionality of our training data.*

In [ ]:
# We identify the hour anchor: Anchor = {"EUR": EUR, "CNY": {"China"}, "JPY": {"Japan"}}

# Going to start by pulling Anchor Countries
# Codes are obtain from yahoo finance symbol
# https://finance.yahoo.com/markets/currencies/

fx_list = {
    # Anchor currencies
    "CNY": "CNY=X",  # China
    "EUR": "EUR=X",  # Euro
    "JPY": "JPY=X",  # Japan
    # Neutral currencies
    "CAD": "CAD=X",  # Canada
    "BRL": "BRL=X",  # Brazil
    "MXN": "MXN=X",  # Mexico
    "COP": "COP=X",  # Colombia
    "PEN": "PEN=X",  # Peru
    "NOK": "NOK=X",  # Norway
    "ZAR": "ZAR=X",  # South Africa
    "INR": "INR=X",  # India
    "TRY": "TRY=X",  # Turkey
    "EGP": "EGP=X",  # Egypt
    "RUB": "RUB=X",  # Russia
    "ILS": "ILS=X",  # Israel
    # CNY group
    "CDF": "CDF=X",  # DR Congo
    "LAK": "LAK=X",  # Laos
    "TZS": "TZS=X",  # Tanzania
    "CLP": "CLP=X",  # Chile
    "GNF": "GNF=X",  # Guinea
    "PKR": "PKR=X",  # Pakistan
    "PHP": "PHP=X",  # Philippines
    "VND": "VND=X",  # Vietnam
    "MRU": "MRU=X",  # Mauritania
    # EUR group
    "ALL": "ALL=X",  # Albania
    "CZK": "CZK=X",  # Czechia
    "TND": "TND=X",  # Tunisia
    "RON": "RON=X",  # Romania
    "HUF": "HUF=X",  # Hungary
    "PLN": "PLN=X",  # Poland
    "RSD": "RSD=X",  # Serbia
    "SEK": "SEK=X",  # Sweden
    "ISK": "ISK=X",  # Iceland
    "DZD": "DZD=X",  # Algeria
    # JPY group
    "PGK": "PGK=X",  # Papua New Guinea
    "TWD": "TWD=X",  # Taiwan
    "THB": "THB=X",  # Thailand
    "AUD": "AUD=X",  # Australia
    "IDR": "IDR=X",  # Indonesia
    "KRW": "KRW=X",  # South Korea
    "MYR": "MYR=X",  # Malaysia
    "NZD": "NZD=X",  # New Zealand
}

# Period is the timeframe we want, we conclude for the time being, it will be 10 years
PERIOD = "10y"

# We conclude we will be looking at exchanges on a daily basis.
INTERVAL = "1d"
FFill_Limit = 3
N_PCS_To_Cluster = 3
N_Cluster = 3

# Creating a list of tickers from the fx_list dictionary
tickers = list(fx_list.values())

# Creating a DataFrame by downloading historical FX data using yfinance
yfinance_fx_raw = yf.download(
    tickers, period=PERIOD, interval=INTERVAL, auto_adjust=None, progress=False
)

In [ ]:
# Remove problem currency
x = yfinance_fx_raw["Close"]
c = x.notna().sum().sort_values()
c

In [ ]:
# We remove currencies that have less than 2000 data points
want = list(fx_list.keys())
t = []
invert = {}

# Loop through each currency in the want list and get their corresponding ticker symbols
for c in want:
    t1, t2 = xxxusd(c)
    t.extend([t1, t2])

# Remove duplicates while preserving order
t = list(dict.fromkeys(t))

# Creating a DataFrame by downloading historical FX data using yfinance
raw = yf.download(
    t, period=PERIOD, interval=INTERVAL, auto_adjust=None, progress=False
)["Close"]

# Creating a DataFrame to hold exchange rate levels and whether they need to be inverted
level = {}
for c in want:
    t1, t2 = xxxusd(c)
    if t1 in raw.columns and raw[t1].notna().sum() > 0:
        level[c] = raw[t1]
        invert[c] = False
    elif t2 in raw.columns and raw[t2].notna().sum() > 0:
        level[c] = 1.0 / raw[t2]
        invert[c] = True

# Creating a DataFrame from the level dictionary and sorting by index
level = pd.DataFrame(level).sort_index()

# Calculating log returns and handling missing values
lreturn = np.log(level / level.shift(1))
lreturn = lreturn.ffill(limit=3).dropna(how="any")

In [ ]:
# Standardizing the log returns
scaler = StandardScaler()
X = pd.DataFrame(
    scaler.fit_transform(lreturn),
    index=lreturn.index,
    columns=lreturn.columns,
)

# PCA and KMeans Clustering
pca = PCA(n_components=2)
Z = pca.fit_transform(X.T)

# Creating a DataFrame for PCA results
pca_df = pd.DataFrame(Z, index=X.columns, columns=["PC1", "PC2"])

# KMeans Clustering
k = 3
km = KMeans(n_clusters=k, random_state=42)
pca_df["cluster"] = km.fit_predict(pca_df[["PC1", "PC2"]].to_numpy())

In [ ]:
# Addidtional pull for explained_variance_ratio_
evr = pca.explained_variance_ratio_
print("Explained variance ratio: ", evr, " | Cumalative: ", evr.cumsum())

In [ ]:
# Grouping currencies based on economic ties
g = {
    "CNY": ["CDF", "LAK", "TZS", "CLP", "GNF", "PKR", "PHP", "VND", "MRU"],
    "EUR": ["ALL", "CZK", "TND", "RON", "HUF", "PLN", "RSD", "SEK", "ISK", "DZD"],
    "JPY": ["PGK", "TWD", "THB", "AUD", "IDR", "KRW", "MYR", "NZD"],
}


# Function to assign group based on currency code
def assign(cc):
    for i, j in g.items():
        if cc == i or cc in j:
            return i
    return "other"


# Assigning group to each currency in the PCA DataFrame
pca_df["group"] = pca_df.index.map(assign)

# Visualization of PCA Results
col = {"CNY": "red", "EUR": "blue", "INR": "green", "JPY": "orange", "other": "gray"}
plt.figure(figsize=(12, 8))
for grp, sub in pca_df.groupby("group"):
    plt.scatter(
        sub["PC1"],
        sub["PC2"],
        c=col.get(grp, "black"),
        s=70,
        alpha=0.85,
        label=f"{grp} (n={len(sub)})",
    )
    for i, j in sub.iterrows():
        plt.annotate(
            i,
            (j["PC1"], j["PC2"]),
            xytext=(3, 3),
            textcoords="offset points",
            fontsize=8,
        )

plt.title("Currencies in PCA Space (standardized returns, XXX/USD)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.show()

In [ ]:
# Calculating Silhouette Score
score = silhouette_score(pca_df[["PC1", "PC2"]], pca_df["cluster"])

# Saving the log returns DataFrame to a Parquet file
lreturn.to_parquet("fx_log_return.parquet", index=False)

## Applying Supervised Learning

#### Grangier Causality

*Applying Grangier causality test to determine if a causal realtionship is found in our anchor and neighboring currency returns*

In [ ]:
# Use this if the above didn't work
# This is a stashed checkpoint file
lreturn = pd.read_parquet("../data/input/fx_log_return.parquet")

# Grouping currencies based on economic ties
g = {
    "CNY": ["CDF", "LAK", "TZS", "CLP", "GNF", "PKR", "PHP", "VND", "MRU"],
    "EUR": ["ALL", "CZK", "TND", "RON", "HUF", "PLN", "RSD", "SEK", "ISK", "DZD"],
    "JPY": ["PGK", "TWD", "THB", "AUD", "IDR", "KRW", "MYR", "NZD"],
}

rdf = granger(lreturn, g, 10, 0.05)
rdf = rdf.sort_values(["anchor", "min_pval"]).reset_index(drop=True)
an = rdf[rdf["significant"] == True].sort_values(["anchor", "min_pval"])
ng = an.groupby("anchor")["neighbor"].apply(list).to_dict()
ng

#### Linear Regression

*Utilizing a linear family model*

In [ ]:
# Importing data
neighbor_groups, df_anchor_neighor, df_log_return_pca = load_data()

# Compiling scores
final = []

# Running LR for each anchor neighbor pair
for option in ["Training_with_Anchor", "Training_without_Anchor"]:

    for i, j in zip(df_anchor_neighor["anchor"], df_anchor_neighor["neighbor"]):

        # Initializing dictionary to hold scores
        scores = {}

        # Preparing data for train test split
        df_1 = df_log_return_pca[[i, j]]
        df_1 = df_1.drop(columns=[i])
        df_1 = create_lag(j, df_1)
        print(df_1.columns)

        # Time-based train-test split
        df_1["target"] = df_1[j].shift(-1)
        df_1.dropna(inplace=True)

        # Define features (X) and target (y)
        if option == "Training_without_Anchor":
            features = [col for col in df_1.columns]
            i = None
        else:
            features = [
                col
                for col in df_1.columns
                if col == j or col.startswith("MA_") or col.startswith("return_lag_")
            ]

        X = df_1[features]
        y = df_1["target"]

        # Splitting data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        clf_lr = LinearRegression().fit(X, y)
        y_preds = clf_lr.predict(X_test)

        # Assigning scores
        scores["anchor"] = i
        scores["neighbor"] = j
        scores["option"] = option
        # scores['rmse'] = root_mean_squared_error(y_test, y_preds)
        scores["rmse"] = np.mean(
            np.sqrt(
                np.abs(
                    cross_val_score(
                        clf_lr, X, y, scoring="neg_mean_squared_error", cv=5, n_jobs=-1
                    )
                )
            )
        )
        scores["model"] = "Linear Regression"
        scores["params"] = None

        # print(scores)
        final.append(scores)

df_final = pd.DataFrame(final)
df_final_summary_mean = df_final.groupby("option")["rmse"].mean()
df_final_summary_std = df_final.groupby("option")["rmse"].std()
print(df_final)
print(df_final_summary_mean)
print(df_final_summary_std)

print(
    rf"Difference without anchor: {df_final_summary_mean[0] - df_final_summary_mean[1]}"
)

#### Extreme Gradient Boosted Regression (XGBoost)

*Utlizing a tree family model*

In [ ]:
# ls_0 = [i for i in ng.items() if i[0] == "EUR"][0]
# ls_1 = [ls_0[0] for i in range(len([i for i in ng.items() if i[0] == "EUR"][0][1]))]
# len(ls_0[1]), len(ls_1)

# xgb_df = pd.DataFrame({"anchor": ls_1, "neighbor": ls_0[1]})

In [ ]:
# # df_0 = pd.read_parquet("../data/input/fx_log_return.parquet")

# for i, j in zip(xgb_df["anchor"], xgb_df["neighbor"]):
#     df_1 = lreturn[[i, j]]
#     df_1.dropna(inplace=True)
#     df_1 = create_lag(j, df_1).drop(columns=[i])
#     score, params = perform_xgb_for_anchor_neighbor(j, df_1)

#     print(f"Anchor: {i}, Neighbor: {j}, Score: {score}, Params: {params}")

## Testing Model Performance

#### Ablation

In [ ]:
# Importing data
neighbor_groups, df_anchor_neighor, df_log_return_pca = load_data()

# Compiling scores
final = []
final_df = pd.DataFrame()

# Preparing data for train test split
df_1 = df_log_return_pca

# List of df_1 columns
# # TODO - remove this filter
# ls_0 = list(create_lag("ALL", df_1[["ALL"]]).columns)

# List of features to ablate
ls_1 = [
    "return_lag_1",
    "return_lag_2",
    "return_lag_3",
    "return_lag_4",
    "return_lag_5",
    "return_lag_6",
    "return_lag_7",
    "return_lag_8",
    "return_lag_9",
    "return_lag_10",
    "return_lag_11",
    "return_lag_12",
    "return_lag_13",
    "return_lag_14",
    "return_lag_15",
    "return_lag_16",
    "return_lag_17",
    "return_lag_18",
    "return_lag_19",
    "return_lag_20",
    "return_lag_21",
    "return_lag_22",
    "return_lag_23",
    "return_lag_24",
    "MA_30",
    "MA_60",
    "MA_90",
]

for ablated in ls_1:
    print(ablated)

    # Looping through each anchor-neighbor pair
    for i, j in zip(df_anchor_neighor["anchor"], df_anchor_neighor["neighbor"]):
        # Initializing dictionary to hold scores
        scores = {}

        # Preparing data for train test split
        df_1 = df_log_return_pca[[i, j]]
        df_1 = df_1.drop(columns=[i])
        df_1 = create_lag(j, df_1)

        ls_0 = list(df_1.columns)
        ls_0.remove(str(ablated))

        df_1 = df_1[ls_0]
        print(df_1.columns)

        # Time-based train-test split
        df_1["target"] = df_1[j].shift(-1)
        df_1.dropna(inplace=True)

        for option in ["Training_with_Anchor", "Training_without_Anchor"]:

            # Define features (X) and target (y)
            if option == "Training_without_Anchor":
                features = [col for col in df_1.columns]
                i = None
            else:
                features = [
                    col
                    for col in df_1.columns
                    if col == j
                    or col.startswith("MA_")
                    or col.startswith("return_lag_")
                ]

            X = df_1[features]
            y = df_1["target"]

            # Splitting data into train and test sets
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=42
            )
            clf_lr = LinearRegression().fit(X, y)
            y_preds = clf_lr.predict(X_test)

            # Assigning scores
            scores["anchor"] = i
            scores["neighbor"] = j
            scores["option"] = option
            # scores['rmse'] = root_mean_squared_error(y_test, y_preds)
            scores["rmse"] = np.mean(
                np.sqrt(
                    np.abs(
                        cross_val_score(
                            clf_lr,
                            X,
                            y,
                            scoring="neg_mean_squared_error",
                            cv=5,
                            n_jobs=-1,
                        )
                    )
                )
            )
            scores["model"] = "Linear Regression"
            scores["params"] = None
            scores["ablated_feature"] = ablated
            scores["columns"] = list(df_1.columns)

            final.append(scores)

        final_df = pd.concat([final_df, pd.DataFrame(final)], ignore_index=True)

final_df.drop_duplicates().to_csv(
    "../data/output/ablation_study_lr_results.csv", sep="|", index=False
)

In [33]:
final_df_summary_mean = final_df.groupby(["option", "ablated_feature"])["rmse"].mean()
final_df_summary_std = final_df.groupby(["option", "ablated_feature"])["rmse"].std()
print(final_df_summary_mean)

option                   ablated_feature
Training_without_Anchor  MA_30              7.389618e-15
                         MA_60              8.887549e-15
                         MA_90              7.964196e-15
                         return_lag_1       1.024440e-14
                         return_lag_10      1.455370e-14
                         return_lag_11      9.289747e-15
                         return_lag_12      1.019660e-14
                         return_lag_13      8.937856e-15
                         return_lag_14      9.706601e-15
                         return_lag_15      1.002450e-14
                         return_lag_16      7.787564e-15
                         return_lag_17      1.022791e-14
                         return_lag_18      9.033822e-15
                         return_lag_19      8.427968e-15
                         return_lag_2       9.217479e-15
                         return_lag_20      1.030380e-14
                         return_lag_21      9.0

In [21]:
!cat ../data/output/ablation_study_lr_results.csv | uniq | sort | uniq | awk -F "|" '{print $1, $2, $3, $4, $7}' | sort | grep "TND" | sort -k4,4

 TND Training_without_Anchor 5.063001039401501e-15 MA_30
 TND Training_without_Anchor 9.07454768883589e-15 MA_60
 TND Training_without_Anchor 5.303004890703234e-15 MA_90
 TND Training_without_Anchor 8.467377817395897e-15 return_lag_1
 TND Training_without_Anchor 3.922554908176262e-15 return_lag_10
 TND Training_without_Anchor 5.470151343387367e-15 return_lag_11
 TND Training_without_Anchor 5.21709313475307e-15 return_lag_12
 TND Training_without_Anchor 5.895988994525267e-15 return_lag_13
 TND Training_without_Anchor 1.0566980352819231e-14 return_lag_14
 TND Training_without_Anchor 1.0066965005746958e-14 return_lag_15
 TND Training_without_Anchor 5.977533061758973e-15 return_lag_16
 TND Training_without_Anchor 6.430274597791554e-15 return_lag_17
 TND Training_without_Anchor 5.678724826339569e-15 return_lag_18
 TND Training_without_Anchor 9.833775463213416e-15 return_lag_19
 TND Training_without_Anchor 4.552751949153748e-15 return_lag_2
 TND Training_without_Anchor 4.33092103973774e-15 r

#### Feature Importance

In [17]:
# Importing data
neighbor_groups, df_anchor_neighor, df_log_return_pca = load_data()

# Compiling scores
final = []

# Running LR for each anchor neighbor pair
for option in ["Training_with_Anchor", "Training_without_Anchor"]:

    for i, j in zip(df_anchor_neighor["anchor"], df_anchor_neighor["neighbor"]):

        # Initializing dictionary to hold scores
        scores = {}

        # Preparing data for train test split
        df_1 = df_log_return_pca[[i, j]]
        df_1 = df_1.drop(columns=[i])
        df_1 = create_lag(j, df_1)

        # Time-based train-test split
        df_1["target"] = df_1[j].shift(-1)
        df_1.dropna(inplace=True)

        # Define features (X) and target (y)
        if option == "Training_without_Anchor":
            features = [col for col in df_1.columns]
            i = None
        else:
            features = [
                col
                for col in df_1.columns
                if col == j or col.startswith("MA_") or col.startswith("return_lag_")
            ]

        X = df_1[features]
        y = df_1["target"]

        # Splitting data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        clf_lr = LinearRegression().fit(X, y)
        y_preds = clf_lr.predict(X_test)

        # Assigning scores
        scores["anchor"] = i
        scores["neighbor"] = j
        scores["option"] = option
        # scores['rmse'] = root_mean_squared_error(y_test, y_preds)
        scores["rmse"] = np.mean(
            np.sqrt(
                np.abs(
                    cross_val_score(
                        clf_lr, X, y, scoring="neg_mean_squared_error", cv=5, n_jobs=-1
                    )
                )
            )
        )
        scores["model"] = "Linear Regression"
        scores["params"] = None
        print(list(zip(clf_lr.coef_, features)))

        # print(scores)
        final.append(scores)

# print(
#     rf"Difference without anchor: {df_final_summary_mean[0] - df_final_summary_mean[1]}"
# )

[(-0.3401070992104422, 'ALL'), (5.671606884949965e-05, 'return_lag_1'), (3.0202383557528296e-06, 'return_lag_2'), (1.1505198454535399e-05, 'return_lag_3'), (1.0564330809437814e-05, 'return_lag_4'), (5.321324062009598e-06, 'return_lag_5'), (6.416562402854899e-05, 'return_lag_6'), (-4.624571087816014e-05, 'return_lag_7'), (-4.05111686481007e-05, 'return_lag_8'), (-2.4054071630962167e-06, 'return_lag_9'), (1.9801335512235516e-05, 'return_lag_10'), (5.5702212274545926e-05, 'return_lag_11'), (-1.7060593771493673e-05, 'return_lag_12'), (-2.044705792315967e-06, 'return_lag_13'), (3.1951824819188976e-06, 'return_lag_14'), (-8.32887531505436e-06, 'return_lag_15'), (3.9566984098646025e-05, 'return_lag_16'), (-9.002890354742378e-06, 'return_lag_17'), (-3.445909636188843e-05, 'return_lag_18'), (3.480521554803273e-05, 'return_lag_19'), (4.7439246336882666e-05, 'return_lag_20'), (-1.4022735485781589e-05, 'return_lag_21'), (-2.5949767828715616e-05, 'return_lag_22'), (-2.6259820960296828e-05, 'return_

## Visualization

In [13]:
pv = rdf.pivot(index="anchor", columns="neighbor", values="min_pval")
pv = pv.fillna(1.0)

plt.figure(figsize=(12, 4 + 0.10 * len(pv.index)))
plt.imshow(pv.values, aspect="auto", cmap="viridis")
plt.xticks(ticks=np.arange(pv.shape[1]), labels=pv.columns, rotation=90)
plt.yticks(ticks=np.arange(pv.shape[0]), labels=pv.index)
plt.colorbar(label="p_value")
plt.title("Granger Causality, Anchor and neighbors")
plt.tight_layout
plt.show()

NameError: name 'rdf' is not defined

## Conclusion